# Radar Chart 

This notebook contains the base code for a radar chart. Base code from the following chart. 

https://gist.github.com/anthonydouc/2ad82f89b4e827f001d786c2b41f4328

In [3]:
import pandas as pd

In [1]:
import pandas as pd
import matplotlib
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import ColumnDataSource, Div, HoverTool, LabelSet
from bokeh.models.glyphs import Ellipse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:


text = ['Health', 'Education', 'Gender', 'Macro', 'Finance', 'PSD', 'Conflict', 'Taxation', 'Democracy']
# example factor:
f1 = np.array([100, 140, 350, 400, 203, 504, 605, 304, 405])
f2 = np.array([60, 530, 540, 330, 200, 406, 709, 403, 509])
f3 = np.array([700, 400, 400, 600 , 320, 670, 302, 609, 304])


# data frame 
df = pd.DataFrame({'text': text, 'f1': f1, 'f2': f2, 'f3': f3})

df.head()

,text,f1,f2,f3
0,Health,100,60,700
1,Education,140,530,400
2,Gender,350,540,400
3,Macro,400,330,600
4,Finance,203,200,320


In [3]:
def long(df, cat_var, value_vars):
    ''' make sure the dataset is in ESDB long format.'''
    # reorder the dataframe is necessary for the rename code to work properly 
    # the wide to long function needs a common begining to each column to be shifted
    
    
    df = df[[cat_var] + value_vars]
    
    # rename columns 
    df.columns = [cat_var] + ['f'+str(f) for f in range(0, len(df.columns[1:]))]
    
    # move from wide to long format
    l = pd.wide_to_long(df, stubnames='f', i=[cat_var], j='var').reset_index()

    
    
    return l

df_plot = long(df, 'text', ['f1', 'f2', 'f3'])
df_plot.head()

,text,var,f
0,Health,0,100
1,Education,0,140
2,Gender,0,350
3,Macro,0,400
4,Finance,0,203


In [11]:
def new_style(p): 
    p.legend.label_text_font_size = '11pt'
    p.legend.background_fill_alpha = 0.3
    p.grid.grid_line_alpha=0.2
    p.title.text_color = '#383951'
    p.title.text_font = "Open Sans Bold"
    p.title.text_font_style = "bold"
    p.grid.grid_line_color='white'
    p.grid.grid_line_width=1.5
    p.axis.axis_label_text_font = "Open Sans Bold"
    p.legend.label_text_font =  "Open Sans Light"
    p.axis.axis_label_text_color = '#999999'
    p.axis.axis_label_text_font_style = 'normal'
    p.xaxis.minor_tick_line_color = None # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None # turn off y-axis minor ticks
    p.axis.axis_line_color=None
    p.axis.major_label_text_font_size = '12pt'
    p.axis.major_tick_line_color=None
    p.axis.minor_tick_line_color=None
    p.axis.major_label_text_color='#999999'
    p.outline_line_color = None
    p.axis.axis_label_text_font_size = '12pt'
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''
    p.yaxis.visible = False  
    p.xaxis.visible = False
    p.title.text_font_size= '30pt'
    
    return p
    
    
    
palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}


def dark_style(p): 
    p.legend.label_text_font_size = '11pt'
    p.legend.background_fill_alpha = 0.3
    p.background_fill_color = '#1B1D24'
    p.grid.grid_line_alpha=0
    p.title.text_color = '#383951'
    p.title.text_font = "Open Sans Bold"
    p.title.text_font_style = "bold"
    p.grid.grid_line_color=''
    p.grid.grid_line_width=1.5
    p.axis.axis_label_text_font = "Open Sans Bold"
    p.legend.label_text_font =  "Open Sans Light"
    p.axis.axis_label_text_color = '#999999'
    p.axis.axis_label_text_font_style = 'normal'
    p.xaxis.minor_tick_line_color = None # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None # turn off y-axis minor ticks
    p.axis.axis_line_color=None
    p.axis.major_label_text_font_size = '12pt'
    p.axis.major_tick_line_color=None
    p.axis.minor_tick_line_color=None
    p.axis.major_label_text_color='#999999'
    p.outline_line_color = None
    p.axis.axis_label_text_font_size = '12pt'
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''
    p.yaxis.visible = False  
    p.xaxis.visible = False
    p.title.text_font_size= '30pt'
    
    p.grid=False
    
    return p
    
    
    
palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}



In [58]:


def radar_plot(df, obs_var, cat_var, value_var, title_text = 'Radar Plot'): 


    output_notebook()

    ###########################################
    ### data input 
    ###########################################
    
    # find min and max values
    max_var = df[value_var].max()
    min_var = df[value_var].min()
    
    

    
    # generate tick marks
    l = matplotlib.ticker.AutoLocator()
    l.create_dummy_axis()
    
    
    axis_values = l.tick_values(min_var, max_var)
    # scale each value by the second to last value for plotting 
    axis_scaled = np.array(axis_values)/axis_values[-2]
    
        
    # generate dictionary of all 
    reals = {}
    scales = {}
    for i in list(df[obs_var].unique()):
        
        reals[str(i)] = df[df[obs_var]==i][value_var].values
        #scale by the largest value in the axis generated by matplotlib
        scales[str(i)] = df[df[obs_var]==i][value_var].values/axis_values[-2]/2
        
        
        
        
    # address if the tick values are too many
    length = len(axis_values) 
    if length>6: 
        ## access every other observatation beginning from the second to last observation moving backwards
        ### add the extra value for consistancy in the labelling from [-1] no matter the length
        axis_values = list(reversed([axis_values[-2+(-2*i)] for i in range(0, int(np.round(len(axis_values)/2)))]))+[1]
        axis_scaled = list(reversed([axis_scaled[-2+(-2*i)] for i in range(0, int(np.round(len(axis_scaled)/2)))]))+[1]
    

    
    

    

    # set values to be ploted
    f1 = scales['0']
    f2 = scales['1']
    f3 = scales['2']
    flist = [f1,f2,f3]
    
    # list of names
    text = list(df[cat_var].unique())



    



    ## generate the number of vars to generate the shape. 
    num_vars = len(text)

    ##############################################
    ######## Functions 

    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta, r):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [0.5, 0.5, r]
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts
    
    # turn data into circlular options
    def radar_patch(r, theta):
        yt = (r) * np.sin(theta) + 0.5
        xt = (r) * np.cos(theta) + 0.5
        return xt, yt
    
    
    
    
    ######### Generate plot
    
    p = figure(title="", plot_width = 740, plot_height= 760, x_range=(-.23,1.2), y_range=(-0.23,1.2)) 
    
    


    #################################
    #### Generate labels 
    #################################
    
    # generate the locations of the labels 
    verts = unit_poly_verts(theta, 0.55)
    x = [v[0] for v in verts] 
    y = [v[1] for v in verts]
    
    # separate between right and left side vars 
    a = (int(np.round(len(text)/2)))+1
    left_labels = text[:a]
    right_labels = text[a:]

    # plot right and left labels (difference -> text align)
    source_left = ColumnDataSource({'x':x[:a],'y':y[:a],'text':left_labels})
    source_right = ColumnDataSource({'x': x[a:]+[0.5],'y':y[a:],'text':right_labels})
    
    label_left = LabelSet(x="x",y="y",text="text",source=source_left, text_font='arial', text_font_size='15pt', 
                     text_color='#9EA6BB', text_align = 'right')
    label_right = LabelSet(x="x",y="y",text="text",source=source_right, text_font='arial', text_font_size='15pt', 
                     text_color='#9EA6BB')
    # add to plot
    p.add_layout(label_left)
    p.add_layout(label_right)
    
    
    
    
    
    
    #################################
    # generate background 
    #################################
    # circles

    for i in axis_scaled:
        glyph = Ellipse(x=0.5, y=0.5, width=i, height=i, fill_color=None, line_color="#9EA6BB", line_alpha=0.5) 
        p.add_glyph(glyph)
    
    #lines - generate coordinates - lines from center to coordinates 
    verts = unit_poly_verts(theta, 0.50)
    x_lines = [v[0] for v in verts] 
    y_lines = [v[1] for v in verts]
    
    for i in range(0,len(x_lines)): 
        p.line(x=(0.5, x_lines[i]), y=(0.5, y_lines[i]), line_width=3, line_color="#9EA6BB", line_alpha=0.5)
    
    #### numbered  
    nums = axis_values[:-1]
    x = np.array(axis_scaled)/2
    x = [.5-i for i in  np.array(axis_scaled)/2]
    y =[0.5, 0.5, 0.5, 0.5, 0.5]

    source = ColumnDataSource({'x':x,'y':y,'text':nums})

    numbers = LabelSet(x="x",y="y",text="text",source=source, text_font='arial', text_font_size='10pt', 
                     text_color='#9EA6BB')

    p.add_layout(numbers)
    
    
    
    
    ##################################
    ####### Plot Patches and circles 
    ###################################
    
    # generate the unique observation variables 
    obs = list(df[obs_var].astype('str').unique())
    
    
    # this sets a maximum numbe of observations at six which is reasonable
    colors = [palette_dark[i] for i in ['USAID Blue', 'USAID Red', 'Medium Blue', 'Light Blue', 'Dark Red', 'Medium Gray']]
    
    
    ##### LINE GRAPH
    for i in range(len(flist)):
        xt, yt = radar_patch(flist[i], theta)

        source = ColumnDataSource()
        p.patch(x=xt, y=yt, fill_alpha=0.3, line_alpha = .9,color=colors[i], line_width = 5, legend=obs[i])
    
    #### CIRCLE Graph 

    sources = {}
    for i in range(0, len(obs)): 
        xt, yt = radar_patch(flist[i], theta)
        sources[i] = {'Category': text, 'Obs': [i]*len(text), 'Value': reals[obs[i]], 'yt': yt, 'xt': xt}


    for i in range(0, len(obs)): 
        s = p.circle(x='xt', y='yt', color = colors[i], source=sources[i],size = 10, fill_alpha=.6)
        hover_circle = HoverTool(
                renderers=[s],
                tooltips=[
                    ('Obs', '@Obs'),
                    ('Category',   '@Category'), 
                    ('Value', '@Value')
                ]
        )
        p.add_tools(hover_circle)

        
    
    ################################
    ######## HOVER
    ################################


    
    #p.tools = None
    #############################
    #### LEGENDs and STYLE
    ##############################
    
    
    p.legend.location = 'bottom_right'
    p.legend.orientation = 'horizontal'

    p.title.text = title_text

    p = dark_style(p)
    p.title.text_font = "Work Sans"
    show(p)
    
    return p

p = radar_plot(df_plot, 'var', 'text', 'f')

Loading BokehJS ...

In [57]:

def dark_style(p): 
    p.legend.label_text_font_size = '11pt'
    p.legend.background_fill_alpha = 0.3
    p.background_fill_color = '#1B1D24'
    p.grid.grid_line_alpha=0
    p.title.text_color = '#9EA6BB'
    p.title.text_font = "Work Sans"
    p.title.text_font_style = "bold"
    p.grid.grid_line_color='white'
    p.grid.grid_line_width=1.5
    p.axis.axis_label_text_font = "Work Sans"
    p.legend.label_text_font =  "Work Sans"
    p.axis.axis_label_text_color = '#999999'
    p.axis.axis_label_text_font_style = 'normal'
    p.xaxis.minor_tick_line_color = None # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None # turn off y-axis minor ticks
    p.axis.axis_line_color=None
    p.axis.major_label_text_font_size = '12pt'
    p.axis.major_tick_line_color=None
    p.axis.minor_tick_line_color=None
    p.axis.major_label_text_color='#999999'
    p.outline_line_color = None
    p.axis.axis_label_text_font_size = '12pt'
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''
    p.yaxis.visible = False  
    p.xaxis.visible = False
    p.title.text_font_size= '30pt'

    p.border_fill_color ='#1B1D24'
    
    
    #####################
    ### Legend settings 
    #####################
    
    p.legend.background_fill_alpha = 0
    p.legend.border_line_alpha = 0.1
    p.legend.label_text_color = '#9EA6BB'
    return p


palette = {'USAID Blue': '#0871F7', 'USAID Red': '#ED3A5D', 'Rich Black': '#B06FD9', 'Medium Blue': '#FFDE44',
    'Light Blue': '#98BEEE', 'Dark Red': '#C10500', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}

